In [9]:
import pandas as pd
import os

root_path = "/Users/kevinlintingwei/YouBike-ETL-Pipeline/data"
processed_path = os.path.join(root_path, "processed")
raw_path = os.path.join(root_path, "raw")

def smart_read_csv(file_path):
    encodings = ['utf-8', 'big5', 'cp950', 'utf-8-sig']
    separators = [',', '\t']
    for enc in encodings:
        for sep in separators:
            try:
                df = pd.read_csv(file_path, encoding=enc, sep=sep)
                # 核心修正 1：立即刪除重複的欄位名稱（標題列層級）
                df = df.loc[:, ~df.columns.duplicated()]
                df.columns = df.columns.str.strip()
                if len(df.columns) > 1:
                    print(f" 成功讀取: {os.path.basename(file_path)} ({enc}, {'Tab' if sep=='\t' else '逗號'})")
                    return df
            except:
                continue
    raise Exception(f" 無法讀取檔案 {file_path}")

def create_tableau_master():
    print(" 啟動 Tableau 專用寬表合成程序...")
    try:
        df_merged = smart_read_csv(os.path.join(processed_path, "youbike_weather_merged.csv"))
        df_info = smart_read_csv(os.path.join(raw_path, "station_info.csv"))
        
        # 核心修正 2：統一更名，但避免產生重複
        rename_dict = {'站點編號': 'station_no', 'station_id': 'station_no', 'id': 'station_no'}
        
        for df in [df_merged, df_info]:
            for old_col, new_col in rename_dict.items():
                if old_col in df.columns and new_col not in df.columns:
                    df.rename(columns={old_col: new_col}, inplace=True)
            # 確保更名後也沒有重複標籤
            df = df.loc[:, ~df.columns.duplicated()]

        # 核心修正 3：精簡 info 表，只取地圖和分析需要的欄位
        # 這樣可以避免 df_merged 和 df_info 有其他同名欄位導致 merge 失敗
        keep_cols = ['station_no', 'station_name', 'lat', 'lng', 'district', 'station_name_en']
        # 只保留存在的欄位
        available_cols = [c for c in keep_cols if c in df_info.columns]
        df_info_clean = df_info[available_cols].drop_duplicates(subset=['station_no'])

        # 確保關鍵字串化
        df_merged['station_no'] = df_merged['station_no'].astype(str)
        df_info_clean['station_no'] = df_info_clean['station_no'].astype(str)
        
        print(f"🔗 正在執行數據關聯 (左表欄位: {len(df_merged.columns)}, 右表欄位: {len(df_info_clean.columns)})...")
        
        # 執行合併
        master_df = pd.merge(df_merged, df_info_clean, on='station_no', how='left')
        
        # 轉換時間
        master_df['record_time'] = pd.to_datetime(master_df['record_time'])
        
        # 輸出
        output_file = os.path.join(root_path, "tableau_master_dataset.csv")
        master_df.to_csv(output_file, index=False, encoding='utf-8-sig')
        
        print("-" * 30)
        print(f" 最終大表包含 {len(master_df.columns)} 個欄位")
        print(f" 檔案路徑: {output_file}")
        
    except Exception as e:
        import traceback
        print(f" 發生錯誤: {e}")
        # traceback.print_exc() # 如果還是失敗，取消這行註解可以看到詳細哪一行報錯

if __name__ == "__main__":
    create_tableau_master()

🚀 啟動 Tableau 專用寬表合成程序...
📖 成功讀取: youbike_weather_merged.csv (utf-8, 逗號)
📖 成功讀取: station_info.csv (utf-8, Tab)
🔗 正在執行數據關聯 (左表欄位: 15, 右表欄位: 4)...
------------------------------
✨ 終於成功了！
📊 最終大表包含 18 個欄位
📍 檔案路徑: /Users/kevinlintingwei/YouBike-ETL-Pipeline/data/tableau_master_dataset.csv
